In [1]:
import cv2
import numpy as np
from tensorflow import keras

In [2]:
# 모델 로드
model = keras.models.load_model("my_model.h5")

## 테스트 이미지 로드  

In [6]:
img = cv2.imread("testdata/man-g4474f6873_1920.jpg")
img.shape

(1279, 1920, 3)

In [7]:
# 이미지 사이즈 조절
img = cv2.resize(img, (1280, 720))
img.shape

(720, 1280, 3)

In [8]:
# 흑백 이미지
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray_img.shape

(720, 1280)

## 얼굴 감지

In [9]:
# 얼굴 감지 객체
face_detector = cv2.CascadeClassifier("haarcascades/haarcascade_frontalface_default.xml")

In [10]:
# faces에 각 얼굴의 좌표가 할당된다.
# x, y, w, h 형태의 좌표
faces = face_detector.detectMultiScale(gray_img, scaleFactor=1.3, minNeighbors=5)
print(f"num_faces: {len(faces)}, faces_roc: {faces}")

num_faces: 1, faces_roc: [[196  67 120 120]]


## 감정 추론 및 이미지화

In [3]:
emotion_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprise"}

In [12]:
# 각 얼굴에 대한 감정을 추론하고 이미지에 표시
for (x, y, w, h) in faces:
    # 얼굴 영역 추출
    face_in_gray_img = gray_img[y:y+h, x:x+w]
    # 차원 수정 (배치 크기, 너비, 높이, 채널) = (1, 48, 48, 1)
    cropped_img = np.expand_dims(np.expand_dims(cv2.resize(face_in_gray_img, (48, 48)), -1), 0)
    # 감정 추론
    predicted_emotion = model.predict(cropped_img)
    
    # 기존 이미지에 박스표시
    cv2.rectangle(img, (x, y-50), (x + w, y + h + 10), (255, 255, 255), 2)
    # 각 감정에 대한 모델 값을 text에 저장
    text = ""
    for i, v in enumerate(predicted_emotion[0]):
        v = round(v * 100)
        text += f"{emotion_dict[i]:}: {v}%\n"
    y0, dy = 50, 25
    for i, line in enumerate(text.split('\n')):
        ny = y0 + i*dy
        cv2.putText(
            img=img,
            text=line,
            org=(x+w+5, ny),
            fontFace=cv2.FONT_HERSHEY_COMPLEX,
            fontScale=0.7,
            color=(255,255,255),
            lineType=cv2.LINE_AA
        )
        
cv2.imshow("hi", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 [==============================] - 0s 26ms/step
